<h3> Init and load in uniform samples </h3>

In [14]:
import numpy as np
import pandas as pd
import json
import yaml
from DeepLearning_Models.utils.general import join, plot_combined
yaml.add_constructor("!join", join)

#THIS IS WHAT IS GOING TO CHANGE BOI
test_file_path = "uniform_samples/BipedalWalker-v3/input_samples.npy"
test_config = "_bipedal"


config_file = open("config_envs/{}.yml".format(test_config))
config = yaml.load(config_file, Loader=yaml.FullLoader)
config.update(yaml.load(open("config_explanations/{}.yml".format(test_config), encoding="utf8"), Loader= yaml.FullLoader))

#read in npy
def read_npy(file_path, config):
    labels = config["picture"]["labels"]
    data = np.load(file_path)
    return pd.DataFrame(data, columns=labels)



<h3> load in old tree version and forward pass </h3>

In [ ]:
testTreeLoc = "SavedCustomTreeMets/treeDTMetsNormd/_bipedal/trees/FalseBaselineFD0.json"
testData = read_npy(test_file_path, config)

with open(testTreeLoc, 'r') as file:
    tree = json.load(file)
    
def ForwardPassOld(tree, row):
    
    if "Value" in tree:
        # We've hit a leaf node
        return tree["Value"]
    
    feature = tree["Feature"]
    bucket = tree["Bucket"]
    
    # Determine whether to go left (True) or right (False)
    if row[feature] < bucket:
        return ForwardPassOld(tree["Left_Child"], row)
    else:
        return ForwardPassOld(tree["Right_Child"], row)

row = testData.iloc[0]
ForwardPassOld(tree, row)


[-0.070003584, -0.27450842, 2.1886513, 1.2663306]

In [ ]:
def forward_pass_with_path(tree, row, path=[]):
    if "Value" in tree:
        return tree["Value"], path
    
    feature = tree["Feature"]
    bucket = tree["Bucket"]
    
    if row[feature] < bucket:
        path.append((feature, "left", bucket))
        return forward_pass_with_path(tree["Left_Child"], row, path)
    else:
        path.append((feature, "right", bucket))
        return forward_pass_with_path(tree["Right_Child"], row, path)

# Get the forward pass and path for the first row
output, path = forward_pass_with_path(tree, row)
print("Output:", output)
print("Path:", path)

def generate_counterfactual(path, row, output, desired_output):
    # Adjust the feature values to affect the first output in the list
    for feature, direction, bucket in reversed(path):
        if direction == "left" and output[0] > desired_output:
            # If we went left, we need to increase the feature to decrease the output
            print(f"Increase {feature} (currently {row[feature]}) closer to or above {bucket} to reduce output")
            row[feature] = bucket + 0.01  # Adjust to go to the other side of the bucket
        elif direction == "right" and output[0] > desired_output:
            # If we went right, we need to decrease the feature to decrease the output
            print(f"Decrease {feature} (currently {row[feature]}) closer to or below {bucket} to reduce output")
            row[feature] = bucket - 0.01  # Adjust to go to the other side of the bucket

    # After adjustments, run forward pass again to check if output decreases
    print("row:",row)
    new_output = forward_pass(tree, row)
    out1, path2 = forward_pass_with_path(tree, row)
    print("Path:", path2)
    return new_output

# Example usage: Decrease the first output to a lower value
desired_output = output[0] - 0.5  # Let's say we want to decrease the first output by 0.5
counterfactual_output = generate_counterfactual(path, df.iloc[0].copy(), output, desired_output)

print("Counterfactual Output:", counterfactual_output)